# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2). Based the original image (not the resized version).

In [1]:
# Load pickled data
import pickle

# TODO: fill this in based on where you saved the training and testing data
training_file = 'traffic-signs-data/train.p'
testing_file = 'traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### To start off let's do a basic data summary.

# TODO: number of training examples
n_train = X_train.shape[0]

# TODO: number of testing examples
n_test = X_test.shape[0]

# TODO: what's the shape of an image?
image_shape = X_train[0].size

# TODO: how many classes are in the dataset
n_classes = np.unique(y_train).size

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = 3072
Number of classes = 43


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

### Import Modules and Setup Global Variables

In [3]:
import tensorflow as tf
import cv2

# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 100

In [4]:
# Image Preprocesing
# We will create more training examples from the data by applying pixel manipulation and image rotation

def do_rotate(img, angle, col_size, row_size):
    M = cv2.getRotationMatrix2D((col_size / 2, row_size / 2), angle, 1)
    return cv2.warpAffine(img, M, (col_size, row_size))


def do_pixel_shift(img, x_shift, y_shift, row_size, col_size):
    M = np.float32([[1, 0, x_shift],[0, 1, y_shift]])    
    return cv2.warpAffine(img, M, (col_size, row_size))


def do_grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


# Training Data Generation
def generate_random_batch(training_data, training_labels, size):
    idx = np.random.choice(training_data.shape[0], size)
    return training_data[idx], training_labels[idx]


# gray_X_train = np.array([do_grayscale(x) for x in X_train])
gray_X_train = X_train
training_image_rowsize = X_train[0].shape[0]
training_image_colsize = X_train[0].shape[1]


rotated_X_train = np.array([do_rotate(x,
                                      np.random.randint(low=-15, high=15),
                                      training_image_colsize,
                                      training_image_rowsize) 
                            for x in gray_X_train])


shifted_X_train = np.array([do_pixel_shift(x,
                                           np.random.randint(low=-2, high=2),
                                           np.random.randint(low=-2, high=2),
                                           training_image_colsize,
                                           training_image_rowsize) 
                            for x in gray_X_train])



In [5]:
n_input = 1024 * 3
n_classes = 43

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.int8, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [6]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

gray_X_train = gray_X_train.reshape((39209, 1024 * 3)).astype('float32')

n_values = (np.max(y_train) + 1)
y_train_ohe = np.eye(n_values)[y_train.astype('int')]

n_values = (np.max(y_test) + 1)
y_test_ohe = np.eye(n_values)[y_test.astype('int')]

In [9]:
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 32, 32, 3])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)
    conv1 = tf.nn.local_response_normalization(conv1)
    
    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    conv2 = tf.nn.local_response_normalization(conv2)
    
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 64], stddev=0.1)),
    'wc2': tf.Variable(tf.random_normal([5, 5, 64, 64], stddev=0.1)),
    'wd1': tf.Variable(tf.random_normal([8*8*64, 256], stddev=0.1)),
    'out': tf.Variable(tf.random_normal([256, n_classes], stddev=0.1))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([64], stddev=0.1)),
    'bc2': tf.Variable(tf.random_normal([64], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([256], stddev=0.1)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.1))
}


# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

training_iters = 200000
init = tf.initialize_all_variables()
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = generate_random_batch(gray_X_train, y_train_ohe, batch_size)

        sess.run(optimizer, feed_dict={x: batch_x, 
                                       y: batch_y,
                                       keep_prob: .75})
        if (step * batch_size) % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, 
                                                        y: batch_y,
                                                        keep_prob: 1.})
            print("Iter " + str(step * batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    
    batch_x, batch_y = X_test.reshape((-1, 32*32*3)), y_test_ohe
    print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={x: batch_x,
                                  y: batch_y,
                                  keep_prob: 1.}))
    

Iter 3200, Minibatch Loss= 3.467214, Training Accuracy= 0.10156
Iter 6400, Minibatch Loss= 2.897506, Training Accuracy= 0.31250
Iter 9600, Minibatch Loss= 2.569930, Training Accuracy= 0.33594
Iter 12800, Minibatch Loss= 1.743386, Training Accuracy= 0.56250
Iter 16000, Minibatch Loss= 1.183058, Training Accuracy= 0.70312
Iter 19200, Minibatch Loss= 1.097394, Training Accuracy= 0.67969
Iter 22400, Minibatch Loss= 0.882539, Training Accuracy= 0.75781
Iter 25600, Minibatch Loss= 0.573447, Training Accuracy= 0.83594
Iter 28800, Minibatch Loss= 0.445397, Training Accuracy= 0.87500
Iter 32000, Minibatch Loss= 0.342689, Training Accuracy= 0.93750
Iter 35200, Minibatch Loss= 0.405580, Training Accuracy= 0.89844
Iter 38400, Minibatch Loss= 0.237127, Training Accuracy= 0.95312
Iter 41600, Minibatch Loss= 0.174692, Training Accuracy= 0.96094
Iter 44800, Minibatch Loss= 0.170882, Training Accuracy= 0.96875
Iter 48000, Minibatch Loss= 0.186248, Training Accuracy= 0.95312
Iter 51200, Minibatch Loss= 

KeyboardInterrupt: 

### Question 1 

_Describe the techniques used to preprocess the data._

In [2]:
### Generate data additional (if you want to!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer:**

In [6]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [5]:
### Train your model here.
### Feel free to use as many code cells as needed.

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [3]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [4]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.